In [1]:
# source /opt/envvars.sh

!pip3 install python-dotenv
%load_ext dotenv
%dotenv -o /opt/envvars.sh
%env

{'HOSTNAME': 'hadoop',
 'OLDPWD': '/',
 'PWD': '/opt',
 'HOME': '/home/hadoop',
 'SHELL': '/bin/bash',
 'SHLVL': '1',
 'PATH': '/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/hadoop/bin:/opt/hadoop/sbin',
 '_': '/usr/bin/nohup',
 'JPY_PARENT_PID': '2960',
 'TERM': 'xterm-color',
 'CLICOLOR': '1',
 'PAGER': 'cat',
 'GIT_PAGER': 'cat',
 'MPLBACKEND': 'module://ipykernel.pylab.backend_inline',
 'JAVA_HOME': '/usr/lib/jvm/java-1.8.0-openjdk-amd64',
 'PDSH_RCMD_TYPE': 'ssh',
 'HADOOP_HOME': '/opt/hadoop',
 'HADOOP_COMMON_HOME': '/opt/hadoop',
 'HADOOP_CONF_DIR': '/opt/hadoop/etc/hadoop',
 'HADOOP_HDFS_HOME': '/opt/hadoop',
 'HADOOP_MAPRED_HOME': '/opt/hadoop',
 'HADOOP_YARN_HOME': '/opt/hadoop'}

# HDFS

## HDFS - Web Interface

- Master node
    - NameNode: http://localhost:9870
    - Secondary NameNode: http://localhost:9868
- Worker node
    - hadoop1
        - DataNode: http://localhost:9864
    - hadoop2
        - DataNode: http://localhost:9865
    - hadoop3
        - DataNode: http://localhost:9866

## HDFS - CLI

In [2]:
%%bash

hdfs help

Usage: hdfs [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]

  OPTIONS is none or any of:

--buildpaths                       attempt to add class files from build tree
--config dir                       Hadoop config directory
--daemon (start|status|stop)       operate on a daemon
--debug                            turn on shell script debug mode
--help                             usage information
--hostnames list[,of,host,names]   hosts to use in worker mode
--hosts filename                   list of hosts to use in worker mode
--loglevel level                   set the log4j level for this command
--workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

cacheadmin           configure the HDFS cache
crypto               configure HDFS encryption zones
debug                run a Debug Admin to execute HDFS debug commands
dfsadmin             run a DFS admin client
dfsrouteradmin       manage Router-based federation
ec                   run a HD

## Filesystem Basic Commands

- https://hadoop.apache.org/docs/r3.2.1/hadoop-project-dist/hadoop-common/FileSystemShell.html

Download books from Gutenberg project (http://www.gutenberg.org/)

- Moby Dick; Or, The Whale by Herman Melville
- Pride and Prejudice by Jane Austen
- Dracula by Bram Stoker

In [3]:
%%bash

cd /opt/datasets

wget -qc http://www.gutenberg.org/files/2701/2701-0.txt -O mobydick.txt
wget -qc http://www.gutenberg.org/files/1342/1342-0.txt -O prideandprejudice.txt
wget -qc http://www.gutenberg.org/cache/epub/345/pg345.txt -O dracula.txt

ls /opt/datasets

dracula.txt
mobydick.txt
prideandprejudice.txt


In [ ]:
%%bash

cd /opt/datasets

# create gutenberg folder in HDFS
# hdfs dfs -mkdir /user/hadoop/gutenberg

# copy books to HDFS
# hdfs dfs -put * /user/hadoop/gutenberg
# hdfs dfs -copyFromLocal * /user/hadoop/gutenberg

# list files in HDFS
# hdfs dfs -ls /user/hadoop/gutenberg

# show first KB of file
# hdfs dfs -head /user/hadoop/gutenberg/mobydick.txt

# show last KB of file
# hdfs dfs -tail /user/hadoop/gutenberg/prideandprejudice.txt

# show whole file - CAREFUL
# hdfs dfs -cat /user/hadoop/gutenberg/dracula.txt

# append file contents to a file in HDFS
# hdfs dfs -appendToFile mobydick.txt prideandprejudice.txt dracula.txt /user/hadoop/allbooks.txt

# copy allbooks.txt (in HDFS) to gutenberg directory (in HDFS)
# hdfs dfs -cp allbooks.txt /user/hadoop/gutenberg
# hdfs dfs -ls -h -R

# retrieve allbooks.txt from HDFS
# hdfs dfs -get allbooks.txt .
# hdfs dfs -copyToLocal /user/hadoop/allbooks.txt .

# remove file
# hdfs dfs -rm allbooks.txt
# hdfs dfs -rm /user/hadoop/allbooks.txt

# mv file (also used for renaming)
# hdfs dfs -mv gutenberg/allbooks.txt gutenberg/books.txt

# print statistics on folder
# printf "name\ttype\tsize\treps\n"
# hdfs dfs -stat "%n %F %b %r" /user/hadoop/gutenberg/*

# getmerge
# hdfs dfs -getmerge /user/hadoop/gutenberg mergebooks.txt

# remove directory and files (-R recursive)
# hdfs dfs -rm -R /user/hadoop/gutenberg

## Utilization in a MapReduce job

In [4]:
%%bash

cd /opt/datasets

hdfs dfs -mkdir /user/hadoop/gutenberg
hdfs dfs -put mobydick.txt prideandprejudice.txt dracula.txt /user/hadoop/gutenberg

In [6]:
%%bash

cd /opt/hadoop/share/hadoop/mapreduce

# run wordcount application
hadoop jar ./hadoop-mapreduce-examples-3.2.2.jar wordcount \
/user/hadoop/gutenberg /user/hadoop/gutenberg-output

2021-06-27 19:56:55,763 INFO client.RMProxy: Connecting to ResourceManager at hadoop/172.17.0.2:8032
2021-06-27 19:56:55,894 INFO client.AHSProxy: Connecting to Application History server at hadoop/172.17.0.2:10200
2021-06-27 19:56:56,088 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/hadoop/.staging/job_1624820343693_0001
2021-06-27 19:56:56,375 INFO input.FileInputFormat: Total input files to process : 3
2021-06-27 19:56:56,494 INFO mapreduce.JobSubmitter: number of splits:3
2021-06-27 19:56:56,645 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1624820343693_0001
2021-06-27 19:56:56,647 INFO mapreduce.JobSubmitter: Executing with tokens: []
2021-06-27 19:56:56,794 INFO conf.Configuration: resource-types.xml not found
2021-06-27 19:56:56,794 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2021-06-27 19:56:57,176 INFO impl.YarnClientImpl: Submitted application application_1624820343693_0001
2021-06-27 19:56

In [7]:
%%bash

# list output folder contents
hdfs dfs -ls /user/hadoop/gutenberg-output

Found 2 items
-rw-r--r--   2 hadoop hadoop          0 2021-06-27 19:57 /user/hadoop/gutenberg-output/_SUCCESS
-rw-r--r--   2 hadoop hadoop     459660 2021-06-27 19:57 /user/hadoop/gutenberg-output/part-r-00000


In [8]:
%%bash

# show head
hdfs dfs -head /user/hadoop/gutenberg-output/part-r-00000

"Defects,"	2
"Information	2
"Plain	4
"Project	10
"Right	2
#1342]	1
#2701]	1
$20,000,000!	1
$5,000)	2
$7,000,000.	1
&	2
'AS-IS',	2
("the	2
($1	2
(1775)	1
(801)	2
(Ahab���s)	1
(Albatross)	1
(American)	1
(Bunger,	1
(Fife)	1
(Greenland	1
(I	1
(Lady	1
(Pig-fish	1
(Pull,	1
(Sperm	3
(Spring,	1
(Steelkilt)	1
(Steelkilt���s)	1
(Strong,	1
(Supplied	3
(Terra	1
(_A	1
(_Advancing_.)	1
(_Ahab	3
(_Algerine	1
(_As	1
(_Ascending,	1
(_Aside_)	1
(_Aside_.)	1
(_Black	1
(_Carpenter	1
(_Dancing_)	1
(_Duodecimo_),	3
(_Duodecimo_).	1
(_During	1
(_Enter	1
(_Fin-Back_).���Under	1
(_Folio_)	1
(_Folio_),	6
(_Foresail	1
(_Grampus_).���Though	1
(_Hump	1
(_Huzza	1
(_Killer_).���Of	1
(_Leaps	1
(_Mealy-mouthed	1
(_Narwhale_),	1
(_Nudging_.)	1
(_Octavo_),	6
(_Octavo_).	1
(_Quietly	1
(_Razor	1
(_Reclining	2
(_Reclining_.)	1
(_Right	1
(_Sings,	1
(_Sperm	1
(_Stubb	1
(_Sulky	1
(_Sulphur	1
(_The	1
(_They	2
(_Thrasher_).���This	1
(_advancing_).	1
(_aside_).	1
(_cabinet_	1
(_grimly_).	1
(_i.e._,	1
(_meeting	1
(_or	1
(_resumin

In [9]:
%%bash

cd /tmp

# copy HDFS file to local filesystem
hdfs dfs -get /user/hadoop/gutenberg-output/part-r-00000 gutenberg-output.txt
head /tmp/gutenberg-output.txt

"Defects,"	2
"Information	2
"Plain	4
"Project	10
"Right	2
#1342]	1
#2701]	1
$20,000,000!	1
$5,000)	2
$7,000,000.	1


In [10]:
%%bash

# remove folder on HDFS
hdfs dfs -rm -R /user/hadoop/gutenberg-output

Deleted /user/hadoop/gutenberg-output


In [11]:
%%bash

cd /opt/hadoop/share/hadoop/mapreduce

# run wordcount application with 2 reducers
hadoop jar ./hadoop-mapreduce-examples-3.2.2.jar wordcount \
-Dmapreduce.job.reduces=2 \
/user/hadoop/gutenberg /user/hadoop/gutenberg-output

2021-06-27 20:14:54,869 INFO client.RMProxy: Connecting to ResourceManager at hadoop/172.17.0.2:8032
2021-06-27 20:14:55,030 INFO client.AHSProxy: Connecting to Application History server at hadoop/172.17.0.2:10200
2021-06-27 20:14:55,241 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/hadoop/.staging/job_1624820343693_0002
2021-06-27 20:14:55,512 INFO input.FileInputFormat: Total input files to process : 3
2021-06-27 20:14:55,663 INFO mapreduce.JobSubmitter: number of splits:3
2021-06-27 20:14:55,825 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1624820343693_0002
2021-06-27 20:14:55,826 INFO mapreduce.JobSubmitter: Executing with tokens: []
2021-06-27 20:14:55,985 INFO conf.Configuration: resource-types.xml not found
2021-06-27 20:14:55,985 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2021-06-27 20:14:56,039 INFO impl.YarnClientImpl: Submitted application application_1624820343693_0002
2021-06-27 20:14

In [12]:
%%bash

# list output folder contents
hdfs dfs -ls /user/hadoop/gutenberg-output

Found 3 items
-rw-r--r--   2 hadoop hadoop          0 2021-06-27 20:15 /user/hadoop/gutenberg-output/_SUCCESS
-rw-r--r--   2 hadoop hadoop     230097 2021-06-27 20:15 /user/hadoop/gutenberg-output/part-r-00000
-rw-r--r--   2 hadoop hadoop     229563 2021-06-27 20:15 /user/hadoop/gutenberg-output/part-r-00001


In [13]:
%%bash

cd /tmp

# copy HDFS file to local filesystem
hdfs dfs -getmerge /user/hadoop/gutenberg-output gutenberg-output.txt
head /tmp/gutenberg-output.txt

hdfs dfs -rm -R /user/hadoop/gutenberg-output

"Project	10
$20,000,000!	1
$7,000,000.	1
'AS-IS',	2
("the	2
($1	2
(1775)	1
(Ahab���s)	1
(American)	1
(Bunger,	1
Deleted /user/hadoop/gutenberg-output


## Advanced Commands

- https://hadoop.apache.org/docs/r3.2.1/hadoop-project-dist/hadoop-hdfs/HDFSCommands.html

### Verify HDFS cluster status

In [14]:
%%bash

# print topology
hdfs dfsadmin -printTopology

printf "\n%40s\n\n" |tr " " "="

hdfs dfsadmin -report

Rack: /default-rack
   172.17.0.3:9866 (hadoop1)
   172.17.0.4:9866 (hadoop2)
   172.17.0.5:9866 (hadoop3)



Configured Capacity: 374878420992 (349.13 GB)
Present Capacity: 49093227958 (45.72 GB)
DFS Remaining: 49086664704 (45.72 GB)
DFS Used: 6563254 (6.26 MB)
DFS Used%: 0.01%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (3):

Name: 172.17.0.3:9866 (hadoop1)
Hostname: hadoop1
Decommission Status : Normal
Configured Capacity: 124959473664 (116.38 GB)
DFS Used: 2030881 (1.94 MB)
Non DFS Used: 102203724511 (95.18 GB)
DFS R

### Replication factor

In [21]:
%%bash

# list folder block location
#hdfs fsck /user/hadoop/gutenberg -files -blocks -locations

# change replication factor of all files in directory to 3
#hdfs dfs -setrep 3 /user/hadoop/gutenberg

# list folder block location
#hdfs fsck /user/hadoop/gutenberg -files -blocks -locations

# change replication factor back to 2
#hdfs dfs -setrep 2 /user/hadoop/gutenberg

# list folder block location
hdfs fsck /user/hadoop/gutenberg -files -blocks -locations

FSCK started by hadoop (auth:SIMPLE) from /172.17.0.2 for path /user/hadoop/gutenberg at Sun Jun 27 20:17:35 GMT 2021

/user/hadoop/gutenberg <dir>
/user/hadoop/gutenberg/dracula.txt 0 bytes, replicated: replication=2, 0 block(s):  OK

/user/hadoop/gutenberg/mobydick.txt 1276233 bytes, replicated: replication=2, 1 block(s):  OK
0. BP-358313605-172.17.0.2-1624820476678:blk_1073741829_1005 len=1276233 Live_repl=2  [DatanodeInfoWithStorage[172.17.0.3:9866,DS-f29f3cb9-1fe3-475e-9530-1cd7805d15c5,DISK], DatanodeInfoWithStorage[172.17.0.4:9866,DS-97d010e6-a95e-40ff-b3b6-ac2cef33bb41,DISK]]

/user/hadoop/gutenberg/prideandprejudice.txt 799645 bytes, replicated: replication=2, 1 block(s):  OK
0. BP-358313605-172.17.0.2-1624820476678:blk_1073741830_1006 len=799645 Live_repl=2  [DatanodeInfoWithStorage[172.17.0.4:9866,DS-97d010e6-a95e-40ff-b3b6-ac2cef33bb41,DISK], DatanodeInfoWithStorage[172.17.0.3:9866,DS-f29f3cb9-1fe3-475e-9530-1cd7805d15c5,DISK]]


Status: HEALTHY
 Number of data-nodes:	3
 Nu

Connecting to namenode via http://hadoop:9870/fsck?ugi=hadoop&files=1&blocks=1&locations=1&path=%2Fuser%2Fhadoop%2Fgutenberg


### Decomission nodes

- dfs.hosts.exclude in hdfs-site.xml

In [22]:
%%bash

# Decomissioning hadoop1
cat > /opt/hadoop/etc/hadoop/dfs.exclude << EOF
hadoop1
EOF

hdfs dfsadmin -refreshNodes

Refresh nodes successful


http://localhost:9870

In [23]:
%%bash

# report HDFS status
hdfs dfsadmin -report

Configured Capacity: 249918947328 (232.76 GB)
Present Capacity: 32725126064 (30.48 GB)
DFS Remaining: 32718848000 (30.47 GB)
DFS Used: 6278064 (5.99 MB)
DFS Used%: 0.02%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (3):

Name: 172.17.0.3:9866 (hadoop1)
Hostname: hadoop1
Decommission Status : Decommissioned
Configured Capacity: 124959473664 (116.38 GB)
DFS Used: 2604425 (2.48 MB)
Non DFS Used: 102205829751 (95.19 GB)
DFS Remaining: 16359395328 (15.24 GB)
DFS Used%: 0.00%
DFS Remaining%: 13.09%
Configured Cache Capacity: 0 

In [24]:
%%bash

# Recomission all nodes
cat > /opt/hadoop/etc/hadoop/dfs.exclude << EOF
EOF

hdfs dfsadmin -refreshNodes

Refresh nodes successful


In [25]:
%%bash

# report HDFS status
hdfs dfsadmin -report

Configured Capacity: 374878420992 (349.13 GB)
Present Capacity: 49086867769 (45.72 GB)
DFS Remaining: 49077985280 (45.71 GB)
DFS Used: 8882489 (8.47 MB)
DFS Used%: 0.02%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (3):

Name: 172.17.0.3:9866 (hadoop1)
Hostname: hadoop1
Decommission Status : Normal
Configured Capacity: 124959473664 (116.38 GB)
DFS Used: 2604425 (2.48 MB)
Non DFS Used: 102205850231 (95.19 GB)
DFS Remaining: 16359374848 (15.24 GB)
DFS Used%: 0.00%
DFS Remaining%: 13.09%
Configured Cache Capacity: 0 (0 B)
Ca

### Handling datanode failures

- timeouts defined in hdfs-site.xml 
    - dfs.namenode.heartbeat.recheck-interval = 10000 (10 seconds)
    - dfs.heartbeat.interval = 3 seconds
- timeout = 2 x recheck-interval + 10 x heartbeat.interval
    - timeout = 50 seconds

In [26]:
%%bash

# get dfs.namenode.heartbeat.recheck-interval
hdfs getconf -confKey dfs.namenode.heartbeat.recheck-interval

# get dfs.heartbeat.interval
hdfs getconf -confKey dfs.heartbeat.interval

10000
3s


In [27]:
%%bash

# simulate node fault
ssh hadoop1 'kill -9 $(cat /tmp/hadoop-hadoop-datanode.pid)'

http://localhost:9870

In [28]:
%%bash

hdfs dfsadmin -report

Configured Capacity: 249918947328 (232.76 GB)
Present Capacity: 32719588683 (30.47 GB)
DFS Remaining: 32713277440 (30.47 GB)
DFS Used: 6311243 (6.02 MB)
DFS Used%: 0.02%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.17.0.4:9866 (hadoop2)
Hostname: hadoop2
Decommission Status : Normal
Configured Capacity: 124959473664 (116.38 GB)
DFS Used: 3170304 (3.02 MB)
Non DFS Used: 102208020480 (95.19 GB)
DFS Remaining: 16356638720 (15.23 GB)
DFS Used%: 0.00%
DFS Remaining%: 13.09%
Configured Cache Capacity: 0 (0 B)
Ca

In [29]:
%%bash

# Restart nodemanager
ssh hadoop1 /opt/hadoop/bin/hdfs --daemon start datanode

In [30]:
%%bash

hdfs dfsadmin -report

Configured Capacity: 374878420992 (349.13 GB)
Present Capacity: 49103261003 (45.73 GB)
DFS Remaining: 49094324224 (45.72 GB)
DFS Used: 8936779 (8.52 MB)
DFS Used%: 0.02%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 7
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (3):

Name: 172.17.0.3:9866 (hadoop1)
Hostname: hadoop1
Decommission Status : Normal
Configured Capacity: 124959473664 (116.38 GB)
DFS Used: 2625536 (2.50 MB)
Non DFS Used: 102200414208 (95.18 GB)
DFS Remaining: 16364789760 (15.24 GB)
DFS Used%: 0.00%
DFS Remaining%: 13.10%
Configured Cache Capacity: 0 (0 B)
Ca